In [26]:
import sys
sys.path.append('..')
import gym

import torch
import torch.nn as nn


from models.td3 import TD3

In [27]:
class Actor(nn.Module):
    def __init__(self, nstates: int, nactions: int):
        super(Actor, self).__init__()
                
        self.fc1 = nn.Linear(nstates, 400)
        self.fc2 = nn.Linear(400, 300)
        self.fc3 = nn.Linear(300, nactions)
        
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
    
    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        
        return x

In [28]:
class Critic(nn.Module): #다름
    def __init__(self, nstates: int, nactions: int):
        super(Critic, self).__init__()
                
        self.fc1 = nn.Linear(nstates + nactions, 400)
        self.fc2 = nn.Linear(400, 300)
        self.fc3 = nn.Linear(300, 1)
        
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
                
    def forward(self, xs):
        x, a = xs
        
        x = self.relu1(self.fc1(torch.cat([x, a], 1)))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [29]:
# import torch.nn.functional as F

# import copy

# from commons.replaybuffer import ReplayBuffer

# class TD3:
#     def __init__(self, env, n_acts: int, act: nn.Module, act_opt: torch.optim, 
#                  cri1: nn.Module, cri1_opt:torch.optim, 
#                  cri2: nn.Module, cri2_opt:torch.optim, 
#                  n_buffer=10000, n_batchs=32, d = 2, gamma = 0.98, tau = 5e-3, eps = 0.3, act_noise = 0.1, noise_range=(-0.5, 0.5), act_range=(-2, 2)):
        
#         self.env = env
#         self.n_acts = n_acts
                
#         self.act = act
#         self.act_opt = act_opt
#         self.act_tg = copy.deepcopy(act)        
        
#         self.cri1 = cri1
#         self.cri1_opt = cri1_opt
#         self.cri1_tg = copy.deepcopy(cri1)
        
#         self.cri2 = cri2
#         self.cri2_opt = cri2_opt
#         self.cri2_tg = copy.deepcopy(cri2)

#         self.buffer = ReplayBuffer(n_buffer)

#         self.n_batchs = n_batchs
        
#         self.gamma = gamma

#         self.act_noise = act_noise
#         self.noise_range = noise_range
#         self.act_range = act_range
        
#         self.d = d
#         self.eps = eps
#         self.tau = tau
                
#     def train(self, n_epis, n_epochs, n_rollout, print_interval=20):
#         env = self.env
        
#         score = 0.0
#         for epi in range(n_epis):
#             s = env.reset()[0]

#             for epoch in range(n_epochs):
#                 for t in range(n_rollout):
#                     a = self.get_action(s)
#                     s_p, r, done, _, _ = env.step(a)
#                     d_mask = 0.0 if done else 1.0
#                     self.buffer.put((s, a, r/100, d_mask, s_p))
#                     env.render()
                    
#                     s = s_p
#                     score += r

#                     if done:
#                         break
                
#                 self.update_cri()
                        
#                 if epoch % self.d == 0:
#                     self.update_act()
            
#                 if epoch % print_interval == 0 and epoch != 0:
#                     print(f"epi: {epoch}, score: {score / print_interval}, n_buffer: {self.buffer.size()}")
#                     score = 0.0
    
        
#     def get_action(self, s):
#         eps = torch.randn(self.n_acts) * self.act_noise
#         a = torch.clamp(self.act(torch.from_numpy(s).float()) + eps, *self.act_range)
#         return a.detach().numpy()
                
#     def update_cri(self):
#         s, a, r, d_mask, s_p = self.buffer.sample(self.n_batchs)
            
#         eps = torch.clamp(torch.randn(self.n_acts) *self.act_noise, *self.noise_range)
#         a_hat = torch.clamp(self.act_tg(s_p) + eps, *self.act_range)

#         y = r + self.gamma * torch.min(self.cri1_tg([s_p, a_hat]), self.cri2_tg([s_p, a_hat])) * d_mask
            
#         cri1_loss = F.mse_loss(self.cri1([s, a]), y.detach())
#         cri2_loss = F.mse_loss(self.cri2([s, a]), y.detach())
                    
#         self.cri1_opt.zero_grad()
#         cri1_loss.backward()
#         self.cri1_opt.step()
            
#         self.cri2_opt.zero_grad()
#         cri2_loss.backward()
#         self.cri2_opt.step()
        
#     def update_act(self):
#         s = self.buffer.sample(self.n_batchs)[0]
        
#         act_loss = - self.cri1([s, self.act(s)]).mean()
#         self.act_opt.zero_grad()
#         act_loss.backward()
#         self.act_opt.step()
        
#         self.update_tg(self.act_tg, self.act)
#         self.update_tg(self.cri1_tg, self.cri1)
#         self.update_tg(self.cri2_tg, self.cri2)
        
#     def update_tg(self, target, source):
#         for tg_params, sr_params in zip(target.parameters(), source.parameters()):
#             tg_params.data.copy_(
#                 sr_params.data * self.tau + (1 - self.tau) * tg_params
#             )

In [30]:
lr = 1e-3
tau = 5e-3
eps = 0
gamma = 0.99

n_sts = 17
n_acts = 6

n_epis = 500
n_epoch = 200
n_rollout = 10

In [31]:
act = Actor(n_sts, n_acts)
act_opt = torch.optim.Adam(act.parameters(), lr=lr)

cri1 = Critic(n_sts, n_acts)
cri_opt1 = torch.optim.Adam(cri1.parameters(), lr=lr)

cri2 = Critic(n_sts, n_acts)
cri_opt2 = torch.optim.Adam(cri2.parameters(), lr=lr)

In [32]:
env = gym.make('HalfCheetah-v4', render_mode='human')

In [33]:
model = TD3(env, n_acts, act, act_opt, cri1, cri_opt1, cri2, cri_opt2, eps=eps, tau=tau, act_range = (-1, 1))

In [34]:
model.train(n_epis, n_epoch, n_rollout)

KeyboardInterrupt: 

In [ ]:
env.close()